In [1]:
import numpy as np
from collab2.foraging.fish import Fish_IndependentRates, animate_trajectories
import pandas as pd
from collab2.foraging.toolkit import (
    animate_predictors,
    dataObject,
    rescale_to_grid,
    subsample_frames_constant_frame_rate,
    derive_predictors_and_scores, plot_coefs
)
from IPython.display import HTML
import matplotlib.pyplot as plt

import copy
import logging
import os
import time
import pyro
import pyro.distributions as dist
import pyro.optim as optim
import seaborn as sns
import torch
from pyro.infer import SVI, Predictive, Trace_ELBO
from pyro.infer.autoguide import AutoMultivariateNormal, init_to_mean
from typing import List
import plotly.express as px

# users can ignore smoke_test -- it's for automatic testing on GitHub, to make sure the notebook runs on future updates to the repository
smoke_test = "CI" in os.environ
num_svi_iters = 10 if smoke_test else 1000
num_samples = 10 if smoke_test else 1000



In [2]:
# choose simulation parameters 
N = 10
arena_size = 150
Tmax = 500
L = 100 
sigma_v = 10
sigma_t = 0.5

# simulate each model

# diffusion only 
interaction_params = {
    "diffusion" :  {
    "rate": 1 * np.ones(N),
    "sigma_v": sigma_v * np.ones(N),
    "sigma_t": sigma_t * np.ones(N),
    }
}

sim_diff = Fish_IndependentRates(
    N=N,
    Tmax=Tmax,
    arena_size=arena_size,
    interaction_params=interaction_params,
)
sim_diff.simulate()


# diffusion + pairwise copying  
interaction_params = {
    "diffusion" :  {
    "rate": 1 * np.ones(N),
    "sigma_v": sigma_v * np.ones(N),
    "sigma_t": sigma_t * np.ones(N),
    },
    "pairwiseCopying" :  {
    "rate": 1 * np.ones(N),
    "interaction_length" : L *np.ones(N),
    "sigma_v": sigma_v * np.ones(N),
    "sigma_t": sigma_t * np.ones(N),
    }
}

sim_pC = Fish_IndependentRates(
    N=N,
    Tmax=Tmax,
    arena_size=arena_size,
    interaction_params=interaction_params,
)
sim_pC.simulate()

# diffusion + vicsek  
interaction_params = {
    "diffusion" :  {
    "rate": 1 * np.ones(N),
    "sigma_v": sigma_v * np.ones(N),
    "sigma_t": sigma_t * np.ones(N),
    },
    "vicsek" :  {
    "rate": 1 * np.ones(N),
    "interaction_length" : L *np.ones(N),
    "sigma_v": sigma_v * np.ones(N),
    "sigma_t": sigma_t * np.ones(N),
    }
}

sim_v = Fish_IndependentRates(
    N=N,
    Tmax=Tmax,
    arena_size=arena_size,
    interaction_params=interaction_params,
)
sim_v.simulate()

In [3]:
# helper function for deriving predictors

def get_foragersDF(sim) : 

    x= sim.trajectories[:,:,0].reshape(-1)
    y =sim.trajectories[:,:,1].reshape(-1)
    num_frames = sim.trajectories.shape[1]
    num_foragers = sim.trajectories.shape[0]
    time = np.tile(np.arange(num_frames), num_foragers)
    foragers = np.concatenate([i * np.ones(num_frames) for i in range(num_foragers)])

    data = {
        "x": x,
        "y": y,
        "time": time,
        "forager": foragers,
    }

    return pd.DataFrame(data)

In [4]:
# choose parameters to discretize grid and subsample
gridMin = -arena_size
gridMax = arena_size
grid_size = 100
conversion_param = grid_size / (2*arena_size)  # used to scale lengths in discrete description
frames_skipped = 2

print(f"length conversion param = {conversion_param}")
# create foragers_objects from each sim

# get DataFrames
foragersDF_diff = get_foragersDF(sim_diff)
foragersDF_pC = get_foragersDF(sim_pC)
foragersDF_v = get_foragersDF(sim_v)

# scaling and subsampling
foragersDF_diff_scaled = rescale_to_grid(
    foragersDF_diff, size=grid_size, gridMin=gridMin, gridMax=gridMax
)
foragersDF_diff_scaled_subsampled = subsample_frames_constant_frame_rate(foragersDF_diff_scaled, frame_spacing=frames_skipped)

foragersDF_pC_scaled = rescale_to_grid(
    foragersDF_pC, size=grid_size, gridMin=gridMin, gridMax=gridMax
)
foragersDF_pC_scaled_subsampled = subsample_frames_constant_frame_rate(foragersDF_pC_scaled, frame_spacing=frames_skipped)

foragersDF_v_scaled = rescale_to_grid(
    foragersDF_v, size=grid_size, gridMin=gridMin, gridMax=gridMax
)
foragersDF_v_scaled_subsampled = subsample_frames_constant_frame_rate(foragersDF_v_scaled, frame_spacing=frames_skipped)

# create foragers objects 
foragers_object_diff = dataObject(
    foragersDF_diff_scaled_subsampled,
    grid_size=grid_size,
)

foragers_object_pC = dataObject(
    foragersDF_pC_scaled_subsampled,
    grid_size=grid_size,
)

foragers_object_v= dataObject(
    foragersDF_v_scaled_subsampled,
    grid_size=grid_size,
)



length conversion param = 0.3333333333333333
original_frames: 5001
original_shape: (50010, 4)
resulting_frames: 2501
resulting_shape: (25010, 4)
original_frames: 5001
original_shape: (50010, 4)
resulting_frames: 2501
resulting_shape: (25010, 4)
original_frames: 5001
original_shape: (50010, 4)
resulting_frames: 2501
resulting_shape: (25010, 4)


In [5]:
# choose kwargs and derive predictors and scores

# define circular grid constraint function 
def circular_constraint_func(grid, c_x, c_y, R):
    ind = ((grid["x"] - c_x) ** 2 + (grid["y"] - c_y) ** 2) < R**2
    return grid.loc[ind]

grid_constraint_params = {
    "c_x": grid_size / 2,
    "c_y": grid_size / 2,
    "R": grid_size / 2,
}

local_windows_kwargs = {
    "window_size":  50,
    "sampling_fraction": 1,
    "skip_incomplete_frames": True,
    "grid_constraint" : circular_constraint_func,
    "grid_constraint_params" : grid_constraint_params
}

sigma_v_p = 10
sigma_t_p = sigma_t

predictor_kwargs = {
    "vicsek" : {
        "dt" : 1,
        "sigma_v" : sigma_v_p,
        "sigma_t" : sigma_t_p,
        "interaction_length" : L * conversion_param

    },
    "pairwiseCopying" : {
        "dt" : 1,
        "sigma_v" : sigma_v_p,
        "sigma_t" : sigma_t_p,
        "interaction_length" : L * conversion_param

    },
    "velocityDiffusion": {
        "dt": 1,
        "sigma_v": sigma_v_p,
        "sigma_t": sigma_t_p,
    }
}

score_kwargs = {
    "nextStep" : {
        "nonlinearity_exponent" : 0.5
    }
}


: 

In [6]:
# derive all predictors and scores
derivedDF_diff = derive_predictors_and_scores(foragers_object_diff, local_windows_kwargs=local_windows_kwargs,predictor_kwargs=predictor_kwargs,score_kwargs=score_kwargs,dropna=True)

2024-09-11 15:09:52,051 - vicsek completed in 86.20 seconds.
2024-09-11 15:12:03,762 - pairwiseCopying completed in 131.71 seconds.
2024-09-11 15:12:45,538 - velocityDiffusion completed in 41.77 seconds.
2024-09-11 15:13:01,004 - nextStep completed in 15.47 seconds.


In [ ]:
derivedDF_pC = derive_predictors_and_scores(foragers_object_pC, local_windows_kwargs=local_windows_kwargs,predictor_kwargs=predictor_kwargs,score_kwargs=score_kwargs,dropna=True)

In [ ]:
derivedDF_v = derive_predictors_and_scores(foragers_object_v, local_windows_kwargs=local_windows_kwargs,predictor_kwargs=predictor_kwargs,score_kwargs=score_kwargs,dropna=True)